<a href="https://colab.research.google.com/github/ulfboge/temporal-landcover-vectorizer/blob/main/scripts/python/combine_rasters_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combine Rasters in Google Colab
This notebook combines raster files by area with each year's data placed in separate bands. The script automatically detects .tif files in the specified directory.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install GDAL in Google Colab
%%capture
!apt-get install -y gdal-bin python3-gdal python3-rasterio
!pip install gdal

In [3]:
import os
from osgeo import gdal
from google.colab import files

# Create directories
data_dir = "/content/drive/MyDrive/earthengine/ndvi"
output_folder = os.path.join(data_dir, "ndvi_combined")

os.makedirs(data_dir, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

In [4]:
# Find and process raster files in data_dir
raster_files = [f for f in os.listdir(data_dir) if f.endswith('.tif')]

if not raster_files:
    print(f"No .tif files found in {data_dir}")
else:
    print(f"Found {len(raster_files)} raster files in {data_dir}:")
    for file in raster_files:
        print(f"  - {file}")

Found 24 raster files in /content/drive/MyDrive/earthengine/ndvi:
  - NDVI_Masked_2013_Area_5.tif
  - NDVI_Masked_2015_Area_5.tif
  - NDVI_Masked_2017_Area_5.tif
  - NDVI_Masked_2019_Area_5.tif
  - NDVI_Masked_2023_Area_5.tif
  - NDVI_Masked_2021_Area_5.tif
  - NDVI_Masked_2013_Area_6.tif
  - NDVI_Masked_2017_Area_6.tif
  - NDVI_Masked_2015_Area_6.tif
  - NDVI_Masked_2019_Area_6.tif
  - NDVI_Masked_2023_Area_6.tif
  - NDVI_Masked_2021_Area_6.tif
  - NDVI_Masked_2015_Area_7.tif
  - NDVI_Masked_2017_Area_7.tif
  - NDVI_Masked_2013_Area_7.tif
  - NDVI_Masked_2021_Area_7.tif
  - NDVI_Masked_2023_Area_7.tif
  - NDVI_Masked_2019_Area_7.tif
  - NDVI_Masked_2015_Area_8.tif
  - NDVI_Masked_2013_Area_8.tif
  - NDVI_Masked_2017_Area_8.tif
  - NDVI_Masked_2019_Area_8.tif
  - NDVI_Masked_2023_Area_8.tif
  - NDVI_Masked_2021_Area_8.tif


In [5]:
# Group files by area
area_files = {}
for raster in raster_files:
    try:
        parts = raster.split("_")
        year = parts[2]  # Extract year
        area_id = parts[-1].replace(".tif", "")  # Extract area ID
        if area_id not in area_files:
            area_files[area_id] = []
        area_files[area_id].append((year, raster))
    except IndexError:
        print(f"Warning: File {raster} does not match expected naming pattern")
        continue

print(f"\nGrouped files by {len(area_files)} areas")


Grouped files by 4 areas


In [6]:
# Process each area
for area_id, files in area_files.items():
    print(f"\nProcessing area {area_id}...")

    # Sort files by year
    files.sort(key=lambda x: x[0])
    print(f"Years included: {[f[0] for f in files]}")

    # Get full file paths
    file_paths = [os.path.join(data_dir, f[1]) for f in files]

    # Open raster datasets
    datasets = [gdal.Open(fp) for fp in file_paths]

    # Check if any dataset is None
    if any(ds is None for ds in datasets):
        print(f"Error: Could not open one or more raster files for area {area_id}.")
        print("File paths:")
        for fp in file_paths:
            print(f"  - {fp}")
        continue  # Skip to the next area

    # Get raster dimensions and projection
    cols = datasets[0].RasterXSize
    rows = datasets[0].RasterYSize
    projection = datasets[0].GetProjection()
    geotransform = datasets[0].GetGeoTransform()

    # Create output file
    output_file = os.path.join(output_folder, f"Combined_Area_{area_id}.tif")
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_file, cols, rows, len(datasets), gdal.GDT_Float32)
    out_ds.SetProjection(projection)
    out_ds.SetGeoTransform(geotransform)

    # Write each year's data to a separate band
    for i, (year, _) in enumerate(files):
        out_band = out_ds.GetRasterBand(i + 1)
        data = datasets[i].GetRasterBand(1).ReadAsArray()
        out_band.WriteArray(data)
        out_band.SetNoDataValue(-9999)
        print(f"  Added year {year} to band {i + 1}")

    # Close datasets
    out_ds = None
    for ds in datasets:
        ds = None

    print(f"Created combined raster: {output_file}")

print(f"\nAll areas processed. Results saved to {output_folder}")


Processing area 5...
Years included: ['2013', '2015', '2017', '2019', '2021', '2023']
  Added year 2013 to band 1
  Added year 2015 to band 2
  Added year 2017 to band 3
  Added year 2019 to band 4
  Added year 2021 to band 5
  Added year 2023 to band 6
Created combined raster: /content/drive/MyDrive/ndvi_combined/Combined_Area_5.tif

Processing area 6...
Years included: ['2013', '2015', '2017', '2019', '2021', '2023']
  Added year 2013 to band 1
  Added year 2015 to band 2
  Added year 2017 to band 3
  Added year 2019 to band 4
  Added year 2021 to band 5
  Added year 2023 to band 6
Created combined raster: /content/drive/MyDrive/ndvi_combined/Combined_Area_6.tif

Processing area 7...
Years included: ['2013', '2015', '2017', '2019', '2021', '2023']
  Added year 2013 to band 1
  Added year 2015 to band 2
  Added year 2017 to band 3
  Added year 2019 to band 4
  Added year 2021 to band 5
  Added year 2023 to band 6
Created combined raster: /content/drive/MyDrive/ndvi_combined/Combined_

In [ ]:
# Zip and download the combined files
!zip -r /content/NDFI_Combined.zip /content/NDFI_Combined
files.download('/content/NDFI_Combined.zip')